In [80]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [81]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [82]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [83]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [84]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [85]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [86]:
image_typographic_reg = linear_model.LinearRegression()

In [87]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [88]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.4920499324944103e-06, 'R2': 0.999999999914808}


In [89]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.13473042845726013, 'R2': 0.5299353711116422}


### Linear Regression: Typographic -> Semantic

In [90]:
typographic_semantic_reg = linear_model.LinearRegression()

In [91]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [92]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.11126150190830231, 'R2': 0.45761648507486485}


In [93]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.11169854551553726, 'R2': 0.4829644128590528}


### Linear Regressions: Image -> Typographic -> Semantic

In [94]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [95]:
train_semantic

array([[0.14613948, 0.13559928, 0.37827376, ..., 0.48616446, 0.62465623,
        0.51961841],
       [0.3715893 , 0.25996411, 0.18734932, ..., 0.83674607, 0.41230209,
        0.44691052],
       [0.31460956, 0.15520793, 0.19810475, ..., 0.74108117, 0.42471779,
        0.47796545],
       ...,
       [0.55167136, 0.56297887, 0.7582385 , ..., 0.16804444, 0.74170125,
        0.68773698],
       [0.32981209, 0.27362459, 0.46467558, ..., 0.52029725, 0.65375805,
        0.53932226],
       [0.74068568, 0.59511034, 0.41721934, ..., 0.63478506, 0.46328393,
        0.32634286]])

In [96]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.11126150190830231, 'R2': 0.45761659018143164}


In [100]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.1292518973350525, 'R2': 0.31130516903960104}


### Linear Regression: Image -> Semantic

In [101]:
image_semantic_reg = linear_model.LinearRegression()

In [102]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [103]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.18795946946193e-06, 'R2': 0.9999999999391834}


In [104]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.10846568644046783, 'R2': 0.5397707651708613}


### Multilayer Perceptron: Image -> Semantic

In [105]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [106]:
image_semantic_nn.fit(train_image, train_semantic)

//anaconda/envs/py229/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [107]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.20766335725784302, 'R2': -1.6755720216397147}


In [108]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.2088954746723175, 'R2': -1.2697851004500285}


#### Example

In [78]:
val_semantic_pred[0]

tensor([1.4112, 1.2048, 0.8896, 1.3990, 0.9943, 1.4014, 1.4787, 1.1337, 1.4718,
        1.0062, 0.9187, 1.0096, 1.3034, 1.0496, 1.1587, 1.5322, 1.2628, 1.0218,
        1.4080, 0.9374, 1.4045, 1.1736, 1.4853, 0.9995, 1.0769, 0.9589, 1.2591,
        1.1337, 1.2856, 1.0522, 1.0452])

In [79]:
data.get_semantic(data.get_name(0, "val"))

array([0.68859958, 0.81998178, 0.55930257, 0.73549266, 0.42331307,
       0.35373612, 0.73091524, 0.81897571, 0.5569037 , 0.8119803 ,
       0.67737495, 0.59392245, 0.81853701, 0.44242379, 0.84911611,
       0.88006872, 0.86581736, 0.77977812, 0.8112426 , 0.47332492,
       0.80354014, 0.80611901, 0.66476793, 0.61109867, 0.66524732,
       0.85484179, 0.22073186, 0.66435573, 0.78214508, 0.52363604,
       0.2637865 ])